In [1]:
import numpy as np # library to handle data in a vectorized manner
!pip install bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
# before clustering we need to normalize data
from sklearn import preprocessing
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 18.3MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                    

In [2]:
address = 'London'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 51.5073219, -0.1276474.


In [2]:
# in case the above cell didn't work
latitude = 51.5073219
longitude = -0.1276474

Let's import the dataframe containing additional information about boroughs

In [35]:
df = pd.read_csv('london-borough-profiles.csv',encoding = 'unicode_escape')
area_names = df['Area_name'].values
df.head()

,Code,Area_name,Inner/_Outer_London,GLA_Population_Estimate_2017,GLA_Household_Estimate_2017,Inland_Area_(Hectares),Population_density_(per_hectare)_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015",Net_internal_migration_(2015),Net_international_migration_(2015),Net_natural_change_(2015),%_of_resident_population_born_abroad_(2015),Largest_migrant_population_by_country_of_birth_(2011),%_of_largest_migrant_population_(2011),Second_largest_migrant_population_by_country_of_birth_(2011),%_of_second_largest_migrant_population_(2011),Third_largest_migrant_population_by_country_of_birth_(2011),%_of_third_largest_migrant_population_(2011),%_of_population_from_BAME_groups_(2016),%_people_aged_3+_whose_main_language_is_not_English_(2011_Census),"Overseas_nationals_entering_the_UK_(NINo),_(2015/16)","New_migrant_(NINo)_rates,_(2015/16)",Largest_migrant_population_arrived_during_2015/16,Second_largest_migrant_population_arrived_during_2015/16,Third_largest_migrant_population_arrived_during_2015/16,Employment_rate_(%)_(2015),Male_employment_rate_(2015),Female_employment_rate_(2015),Unemployment_rate_(2015),Youth_Unemployment_(claimant)_rate_18-24_(Dec-15),Proportion_of_16-18_year_olds_who_are_NEET_(%)_(2014),Proportion_of_the_working-age_population_who_claim_out-of-work_benefits_(%)_(May-2016),%_working-age_with_a_disability_(2015),Proportion_of_working_age_people_with_no_qualifications_(%)_2015,Proportion_of_working_age_with_degree_or_equivalent_and_above_(%)_2015,"Gross_Annual_Pay,_(2016)",Gross_Annual_Pay_-_Male_(2016),Gross_Annual_Pay_-_Female_(2016),Modelled_Household_median_income_estimates_2012/13,%_adults_that_volunteered_in_past_12_months_(2010/11_to_2012/13),Number_of_jobs_by_workplace_(2014),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),Crime_rates_per_thousand_population_2014/15,Fires_per_thousand_population_(2014),Ambulance_incidents_per_hundred_population_(2014),"Median_House_Price,_2015","Average_Band_D_Council_Tax_charge_(£),_2015/16",New_Homes_(net)_2015/16_(provisional),"Homes_Owned_outright,_(2014)_%","Being_bought_with_mortgage_or_loan,_(2014)_%","Rented_from_Local_Authority_or_Housing_Association,_(2014)_%","Rented_from_Private_landlord,_(2014)_%","%_of_area_that_is_Greenspace,_2005",Total_carbon_emissions_(2014),"Household_Waste_Recycling_Rate,_2014/15","Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)","%_of_adults_who_cycle_at_least_once_per_month,_2014/15","Average_Public_Transport_Accessibility_score,_2014","Achievement_of_5_or_more_A*-_C_grades_at_GCSE_or_equivalent_including_English_and_Maths,_2013/14",Rates_of_Children_Looked_After_(2016),%_of_pupils_whose_first_language_is_not_English_(2015),%_children_living_in_out-of-work_households_(2015),"Male_life_expectancy,_(2012-14)","Female_life_expectancy,_(2012-14)",Teenage_conception_rate_(2014),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),Childhood_Obesity_Prevalance_(%)_2015/16,People_aged_17+_with_diabetes_(%),Mortality_rate_from_causes_considered_preventable_2012/14,Political_control_in_council,Proportion_of_seats_won_by_Conservatives_in_2014_election,Proportion_of_seats_won_by_Labour_in_2014_election,Proportion_of_seats_won_by_Lib_Dems_in_2014_election,Turnout_at_2014_local_elections
0,E09000001,City of London,Inner London,8800,5326,290,30.3,43.2,11.4,73.1,15.5,-7,665,30,.,United States,2.8,France,2,Australia,1.9,27.5,17.1,975,152.2,India,France,United States,64.6,.,.,.,1.6,.,3.4,.,.,.,.,.,.,"£63,620",.,500400,3.4,84.3,26130,64.3,.,12.3,.,799999,931.2,80,.,.,.,.,4.8,1036,34.4,1692,0.4,16.9,7.9,78.6,101,.,7.9,.,.,.,6.6,7.1,6.0,5.6,NaN,2.6,129,.,.,.,.,.
1,E09000002,Barking and Dagenham,Outer London,209000,

Let's parse the web to get the latitudes and longitudes of each Area_name because we don't have them in this dataset

In [22]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_London_boroughs').text
soup = BeautifulSoup(source,'html.parser')

In [23]:
# There are two tables we need in this page
table = soup.find_all('table')
print(len(table))

5


In [24]:
# By looking at the page we only need the first two tables
boroughs = table[0]
city_of_london = table[1]

In [25]:
boroughs_area_names = [x.contents for x in boroughs.find_all('a') if x.contents in area_names]
city_of_london_name = [x.contents for x in city_of_london.find_all('a') if x.contents in area_names][0]
print(city_of_london_name)
boroughs_area_names.append(city_of_london_name)
list_of_names = [x[0] for x in boroughs_area_names]
print(list_of_names)
print(len(list_of_names))

['City of London']
['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley', 'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney', 'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham', 'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton', 'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster', 'City of London']
33


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  


In [26]:
# Now that we have all area names, we will get their respective coordinates
boroughs_geo = [x.contents for x in boroughs.find_all('span', {"class": "geo"})]
city_of_london_geo = [x.contents for x in city_of_london.find_all('span', {"class": "geo"})][0]
print(city_of_london_geo)
boroughs_geo.append(city_of_london_geo)
print(boroughs_geo)

['51.5155; -0.0922']
[['51.5607; 0.1557'], ['51.6252; -0.1517'], ['51.4549; 0.1505'], ['51.5588; -0.2817'], ['51.4039; 0.0198'], ['51.5290; -0.1255'], ['51.3714; -0.0977'], ['51.5130; -0.3089'], ['51.6538; -0.0799'], ['51.4892; 0.0648'], ['51.5450; -0.0553'], ['51.4927; -0.2339'], ['51.6000; -0.1119'], ['51.5898; -0.3346'], ['51.5812; 0.1837'], ['51.5441; -0.4760'], ['51.4746; -0.3680'], ['51.5416; -0.1022'], ['51.5020; -0.1947'], ['51.4085; -0.3064'], ['51.4607; -0.1163'], ['51.4452; -0.0209'], ['51.4014; -0.1958'], ['51.5077; 0.0469'], ['51.5590; 0.0741'], ['51.4479; -0.3260'], ['51.5035; -0.0804'], ['51.3618; -0.1945'], ['51.5099; -0.0059'], ['51.5908; -0.0134'], ['51.4567; -0.1910'], ['51.4973; -0.1372'], ['51.5155; -0.0922']]


In [27]:
# Now we will separate the coordinates into two columns : latitude and longitude
latitudes, longitudes = [], []
for g in boroughs_geo:
    split = g[0].split('; ')
    latitudes.append(split[0])
    longitudes.append(split[1])
print(latitudes)

['51.5607', '51.6252', '51.4549', '51.5588', '51.4039', '51.5290', '51.3714', '51.5130', '51.6538', '51.4892', '51.5450', '51.4927', '51.6000', '51.5898', '51.5812', '51.5441', '51.4746', '51.5416', '51.5020', '51.4085', '51.4607', '51.4452', '51.4014', '51.5077', '51.5590', '51.4479', '51.5035', '51.3618', '51.5099', '51.5908', '51.4567', '51.4973', '51.5155']


Finally, let's create the dataframe

In [28]:
geo_df = pd.DataFrame({'Area_name':list_of_names,'latitude':latitudes,'longitude':longitudes})
geo_df.head()

,Area_name,latitude,longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198


Let's finally merge the two dataframe into one

In [36]:
merged_df = pd.merge(df,geo_df,'left',on='Area_name')
merged_df.head()

,Code,Area_name,Inner/_Outer_London,GLA_Population_Estimate_2017,GLA_Household_Estimate_2017,Inland_Area_(Hectares),Population_density_(per_hectare)_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015",Net_internal_migration_(2015),Net_international_migration_(2015),Net_natural_change_(2015),%_of_resident_population_born_abroad_(2015),Largest_migrant_population_by_country_of_birth_(2011),%_of_largest_migrant_population_(2011),Second_largest_migrant_population_by_country_of_birth_(2011),%_of_second_largest_migrant_population_(2011),Third_largest_migrant_population_by_country_of_birth_(2011),%_of_third_largest_migrant_population_(2011),%_of_population_from_BAME_groups_(2016),%_people_aged_3+_whose_main_language_is_not_English_(2011_Census),"Overseas_nationals_entering_the_UK_(NINo),_(2015/16)","New_migrant_(NINo)_rates,_(2015/16)",Largest_migrant_population_arrived_during_2015/16,Second_largest_migrant_population_arrived_during_2015/16,Third_largest_migrant_population_arrived_during_2015/16,Employment_rate_(%)_(2015),Male_employment_rate_(2015),Female_employment_rate_(2015),Unemployment_rate_(2015),Youth_Unemployment_(claimant)_rate_18-24_(Dec-15),Proportion_of_16-18_year_olds_who_are_NEET_(%)_(2014),Proportion_of_the_working-age_population_who_claim_out-of-work_benefits_(%)_(May-2016),%_working-age_with_a_disability_(2015),Proportion_of_working_age_people_with_no_qualifications_(%)_2015,Proportion_of_working_age_with_degree_or_equivalent_and_above_(%)_2015,"Gross_Annual_Pay,_(2016)",Gross_Annual_Pay_-_Male_(2016),Gross_Annual_Pay_-_Female_(2016),Modelled_Household_median_income_estimates_2012/13,%_adults_that_volunteered_in_past_12_months_(2010/11_to_2012/13),Number_of_jobs_by_workplace_(2014),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),Crime_rates_per_thousand_population_2014/15,Fires_per_thousand_population_(2014),Ambulance_incidents_per_hundred_population_(2014),"Median_House_Price,_2015","Average_Band_D_Council_Tax_charge_(£),_2015/16",New_Homes_(net)_2015/16_(provisional),"Homes_Owned_outright,_(2014)_%","Being_bought_with_mortgage_or_loan,_(2014)_%","Rented_from_Local_Authority_or_Housing_Association,_(2014)_%","Rented_from_Private_landlord,_(2014)_%","%_of_area_that_is_Greenspace,_2005",Total_carbon_emissions_(2014),"Household_Waste_Recycling_Rate,_2014/15","Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)","%_of_adults_who_cycle_at_least_once_per_month,_2014/15","Average_Public_Transport_Accessibility_score,_2014","Achievement_of_5_or_more_A*-_C_grades_at_GCSE_or_equivalent_including_English_and_Maths,_2013/14",Rates_of_Children_Looked_After_(2016),%_of_pupils_whose_first_language_is_not_English_(2015),%_children_living_in_out-of-work_households_(2015),"Male_life_expectancy,_(2012-14)","Female_life_expectancy,_(2012-14)",Teenage_conception_rate_(2014),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),Childhood_Obesity_Prevalance_(%)_2015/16,People_aged_17+_with_diabetes_(%),Mortality_rate_from_causes_considered_preventable_2012/14,Political_control_in_council,Proportion_of_seats_won_by_Conservatives_in_2014_election,Proportion_of_seats_won_by_Labour_in_2014_election,Proportion_of_seats_won_by_Lib_Dems_in_2014_election,Turnout_at_2014_local_elections,latitude,longitude
0,E09000001,City of London,Inner London,8800,5326,290,30.3,43.2,11.4,73.1,15.5,-7,665,30,.,United States,2.8,France,2,Australia,1.9,27.5,17.1,975,152.2,India,France,United States,64.6,.,.,.,1.6,.,3.4,.,.,.,.,.,.,"£63,620",.,500400,3.4,84.3,26130,64.3,.,12.3,.,799999,931.2,80,.,.,.,.,4.8,1036,34.4,1692,0.4,16.9,7.9,78.6,101,.,7.9,.,.,.,6.6,7.1,6.0,5.6,NaN,2.6,129,.,.,.,.,.,51.5155,-0.0922
1,E09000002,Barkin

In [37]:
# Careful when looking at the end of the dataframe we see statistics about area that are not boroughs -> United-Kingdom...
# We need to drop those rows as we will only focus on boroughs
print(merged_df.tail())
merged_df = merged_df.drop([33,34,35,36,37])

         Code       Area_name Inner/_Outer_London  \
33  E13000001    Inner London                 NaN   
34  E13000002    Outer London                 NaN   
35  E12000007          London                 NaN   
36  E92000001         England                 NaN   
37  K02000001  United Kingdom                 NaN   

    GLA_Population_Estimate_2017 GLA_Household_Estimate_2017  \
33                       3535700                     1522541   
34                       5299800                     2079422   
35                       8835500                     3601963   
36                      55609600                           .   
37                      65999100                           .   

   Inland_Area_(Hectares) Population_density_(per_hectare)_2017  \
33                 31,929                                 110.7   
34                125,424                                  42.3   
35                157,215                                  56.2   
36             13,025,967   

Some preprocessing is necessary as we can see missing values in some columns

In [38]:
# We first need to replace missing field ' ' as np.nan because we don't know how much missing row there is for one column
for column in merged_df.columns:
    merged_df[column] = merged_df[column].replace(to_replace=['.'],value=np.nan)
merged_df.isnull().sum()

Code                                                                                                 0
Area_name                                                                                            0
Inner/_Outer_London                                                                                  0
GLA_Population_Estimate_2017                                                                         0
GLA_Household_Estimate_2017                                                                          0
Inland_Area_(Hectares)                                                                               0
Population_density_(per_hectare)_2017                                                                0
Average_Age,_2017                                                                                    0
Proportion_of_population_aged_0-15,_2015                                                             0
Proportion_of_population_of_working-age,_2015                            

In [39]:
# We select features that does not contain null values
features = [c for c in merged_df.columns if merged_df[c].isnull().sum()==0]
#print(features)
print(merged_df[features].info())
# we can see latitude and longitude defined as type object so let's convert them to float
merged_df['latitude'] = merged_df['latitude'].astype('float')
merged_df['longitude'] = merged_df['longitude'].astype('float')
non_object_columns = np.append(merged_df[features].select_dtypes(exclude=[object]).columns.values,'Area_name')
print(merged_df[non_object_columns].info())
df_filtered = merged_df[non_object_columns]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 0 to 32
Data columns (total 58 columns):
Code                                                                                                33 non-null object
Area_name                                                                                           33 non-null object
Inner/_Outer_London                                                                                 33 non-null object
GLA_Population_Estimate_2017                                                                        33 non-null int64
GLA_Household_Estimate_2017                                                                         33 non-null object
Inland_Area_(Hectares)                                                                              33 non-null object
Population_density_(per_hectare)_2017                                                               33 non-null object
Average_Age,_2017                                                

In [40]:
df_filtered.head()

,GLA_Population_Estimate_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015","New_migrant_(NINo)_rates,_(2015/16)",Employment_rate_(%)_(2015),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),"Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)",%_children_living_in_out-of-work_households_(2015),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),People_aged_17+_with_diabetes_(%),latitude,longitude,Area_name
0,8800,43.2,11.4,73.1,15.5,152.2,64.6,3.4,84.3,26130,64.3,1692,0.4,7.9,6.6,7.1,6.0,5.6,2.6,51.5155,-0.0922,City of London
1,209000,32.9,27.2,63.1,9.7,59.1,65.8,21.1,0.5,6560,73.0,56966,0.8,18.7,7.1,7.6,7.1,3.1,7.3,51.5607,0.1557,Barking and Dagenham
2,389600,37.3,21.1,64.9,14.0,53.1,68.5,18.7,0.7,26190,73.8,144717,1.1,9.3,7.5,7.8,7.4,2.8,6.0,51.6252,-0.1517,Barnet
3,244300,39.0,20.6,62.9,16.6,14.4,75.1,15.9,0.6,9075,73.5,108507,1.2,12.6,7.4,7.7,7.2,3.3,6.9,51.4549,0.1505,Bexley
4,332100,35.6,20.9,67.8,11.3,100.9,69.5,17.6,0.6,15745,74.4,87802,0.8,13.7,7.3,7.4,7.2,2.9,7.9,51.5588,-0.2817,Brent


In [41]:
df_filtered.to_csv('df_filtered.csv',index=False)

### Using Foursquare API

In [2]:
CLIENT_ID = 'YBG5BPNALNDJINTSYHR4DEMXI3PYEW1SEPZ5O2RFJYDZ15SA' # your Foursquare ID
CLIENT_SECRET = 'ABO4REOHM0CK5VJCJI4BHH51AYMXEVMRFE42NKHWXUXCYZHW' # your Foursquare Secret
VERSION = '20191105'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YBG5BPNALNDJINTSYHR4DEMXI3PYEW1SEPZ5O2RFJYDZ15SA
CLIENT_SECRET:ABO4REOHM0CK5VJCJI4BHH51AYMXEVMRFE42NKHWXUXCYZHW


In [3]:
def getNearbyRestaurantVenues(name, lat, lng, offset, radius=2000):
    venues_list=[]          
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section=food&offset={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            offset,
            LIMIT)
    # make the GET request
    results = []
    #print(requests.get(url).json()["response"])
    if('groups' in requests.get(url).json()["response"]):
        print(requests.get(url).json()['meta'])
        results = requests.get(url).json()["response"]['groups'][0]['items']
    #print(len(results))#
    for v in results:
        if('Restaurant' in v['venue']['categories'][0]['name']):
            #print(v['venue']['categories'][0]['name'])
            # return only relevant information for each nearby venue
            venues_list.append([(
                    v['venue']['id'],
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name'])])
    #print(len(venues_list))
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    if(len(nearby_venues)!=0):
        nearby_venues.columns = ['Id',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    #print(len(nearby_venues))
    return(nearby_venues)
def countNearbyVenues(lat,lng,radius=2000):
    # We count the nb of non restaurant venues near the current venue in a radius equal to 200m
    venues_list=[]          
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    # make the GET request
    results = []
    if('groups' in requests.get(url).json()["response"]):
        results = requests.get(url).json()["response"]['groups'][0]['items']
    count = 0
    for v in results:
        if('Restaurant' not in v['venue']['categories'][0]['name']): count+=1
    return count
def countNearbyRestaurantVenues(lat,lng,radius=2000):
    # We count the nb of non restaurant venues near the current venue in a radius equal to 200m
    venues_list=[]          
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section=food&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    # make the GET request
    results = []
    if('groups' in requests.get(url).json()["response"]):
        results = requests.get(url).json()["response"]['groups'][0]['items']
    count = 0
    for v in results:
        if('Restaurant' in v['venue']['categories'][0]['name']): count+=1
    return count

In [24]:
# Since the call from the API is limited to 100 results we need to iterate over it to get a consistent amount of data
# We also need to slightly modify lat/long because otherwise it will return the same 100 results
# We will aim for around 1000 unique restaurants in London
borough_venues = []
for idx,r in df_filtered.iterrows():
    borough_venues.append(getNearbyRestaurantVenues(name=r['Area_name'],
                                       lat=r['latitude'],
                                       lng=r['longitude'],
                                       offset=0))
#print(borough_venues)
# While debugging, I got this error so this may explains why the code sometimes doesn't work, just need to relaunch it
# {'code': 500, 'errorType': 'server_error', 'errorDetail': 'Foursquare servers are experiencing problems. Please retry and check status.foursquare.com for updates.'}
venues = []
for idx,k in enumerate(borough_venues):
    b_venues = k
    for j in range(10):
        for i in range(0,3):
            tmp = getNearbyRestaurantVenues(name=df_filtered.iloc[idx]['Area_name'],
                                           lat=df_filtered.iloc[idx]['latitude']+0.0008983*j,
                                           lng=df_filtered.iloc[idx]['longitude']+0.0008983*j,
                                           offset=100*i)
        b_venues = pd.concat([b_venues,tmp]).drop_duplicates().reset_index(drop=True)
    venues.append(b_venues)
    print(b_venues.shape)

{'code': 200, 'requestId': '5dc28e9abcbf7a00398c76ec'}
{'code': 200, 'requestId': '5dc28e9bd03d29002b0832b4'}
{'code': 200, 'requestId': '5dc28e9c492822002de4bc25'}
{'code': 500, 'errorType': 'server_error', 'errorDetail': 'Foursquare servers are experiencing problems. Please retry and check status.foursquare.com for updates.'}
{'code': 200, 'requestId': '5dc28ea96bdee6003929b002'}
{'code': 200, 'requestId': '5dc28eaaa6ec98002c727a8b'}
{'code': 200, 'requestId': '5dc28e906b9b4900204e64cc'}
{'code': 200, 'requestId': '5dc28e2b02a172002890f747'}
{'code': 200, 'requestId': '5dc28ea160ba080029caa38f'}
{'code': 200, 'requestId': '5dc28eae89b06a002c697f11'}
{'code': 200, 'requestId': '5dc28eb0d9a6e6002b2f7284'}
{'code': 200, 'requestId': '5dc28eb12b274a003947e8b3'}
{'code': 200, 'requestId': '5dc28eb3342adf00383f770c'}
{'code': 200, 'requestId': '5dc28eb4bcbf7a00398ce21e'}
{'code': 200, 'requestId': '5dc28e996b9b4900204e8ba6'}
{'code': 200, 'requestId': '5dc28eb5d03360002c62d5a8'}
{'code': 2

In [37]:
final_venues = pd.DataFrame()
for idx,v in enumerate(venues):
    if(len(v)!=0):
        v['Area_name'] = df_filtered.iloc[idx]['Area_name']
        final_venues = pd.concat([final_venues,v]).drop_duplicates().reset_index(drop=True)
print(final_venues.shape)
final_venues.head()

(1829, 6)


,Id,Venue,Venue Latitude,Venue Longitude,Venue Category,Area_name
0,4cadc734632b370400f7696e,City Càphê,51.514750,-0.091545,Vietnamese Restaurant,City of London
1,4dff32bae4cdf7246077a9aa,Pilpel,51.515195,-0.098462,Falafel Restaurant,City of London
2,4ac518d5f964a5200fa820e3,Coq d'Argent,51.513303,-0.091036,French Restaurant,City of London
3,4af89520f964a520710e22e3,Le Relais de Venise,51.514727,-0.086527,French Restaurant,City of London
4,50ed6e85e4b0ffbaac00836c,Burger & Lobster,51.513687,-0.094643,Seafood Restaurant,City of London


In [61]:
count_df = final_venues.groupby('Area_name').size().reset_index()
count_df.rename(columns={0:'Restaurant Count'},inplace=True)
count_df.head()

,Area_name,Restaurant Count
0,Barking and Dagenham,3
1,Barnet,31
2,Brent,41
3,Bromley,21
4,Camden,183


In [63]:
final_venues = pd.merge(final_venues,count_df,how='left',on='Area_name')

In [69]:
# We can see that we have several same venue with same category but different latitude and longitude so we will group them together
# and get the mean latitude and longitude
names, lat, long, cat, count, neighborhood = [],[],[],[],[],[]
for r in final_venues['Venue'].unique():
    names.append(r)
    cat.append(final_venues.loc[final_venues['Venue']==r,:]['Venue Category'].values[0])
    lat.append(final_venues.loc[final_venues['Venue']==r,:].mean(axis=0)['Venue Latitude'])
    long.append(final_venues.loc[final_venues['Venue']==r,:].mean(axis=0)['Venue Longitude'])
    neighborhood.append(final_venues.loc[final_venues['Venue']==r,:]['Area_name'].values[0])
    count.append(final_venues.loc[final_venues['Venue']==r,:]['Restaurant Count'].values[0])
neighbor_df = pd.DataFrame({'area_name':names,'lat':lat,'long':long,'cat':cat,'restaurant count':count,'borough':neighborhood})
neighbor_df.head()
print(neighbor_df.shape)

(1378, 6)


In [70]:
# let's group the restaurants of a borough with one hot encoding

In [71]:
neighbor_df.head()

,area_name,lat,long,cat,restaurant count,borough
0,City Càphê,51.514750,-0.091545,Vietnamese Restaurant,135,City of London
1,Pilpel,51.513738,-0.096216,Falafel Restaurant,135,City of London
2,Coq d'Argent,51.513303,-0.091036,French Restaurant,135,City of London
3,Le Relais de Venise,51.514727,-0.086527,French Restaurant,135,City of London
4,Burger & Lobster,51.513084,-0.116645,Seafood Restaurant,135,City of London


In [72]:
neighbor_df.to_csv('restaurants.csv',index=False)

In [4]:
neighbor_df = pd.read_csv('restaurants.csv')
neighbor_df.head()

,area_name,lat,long,cat,restaurant count,borough
0,City Càphê,51.514750,-0.091545,Vietnamese Restaurant,135,City of London
1,Pilpel,51.513738,-0.096216,Falafel Restaurant,135,City of London
2,Coq d'Argent,51.513303,-0.091036,French Restaurant,135,City of London
3,Le Relais de Venise,51.514727,-0.086527,French Restaurant,135,City of London
4,Burger & Lobster,51.513084,-0.116645,Seafood Restaurant,135,City of London


In [15]:
# We will store the frequency of each category of restaurant in a borough
boroughs = [x for x in list(neighbor_df['borough'].unique())]
freq = {}
for b in boroughs:
    freq[b] = neighbor_df[neighbor_df.borough==b].groupby('cat').size()
print(freq)

{'City of London': cat
Afghan Restaurant                 1
American Restaurant               4
Arepa Restaurant                  1
Argentinian Restaurant            1
Asian Restaurant                  3
Austrian Restaurant               1
Cantonese Restaurant              1
Caucasian Restaurant              1
Chinese Restaurant                1
Eastern European Restaurant       1
Empanada Restaurant               1
English Restaurant                4
Falafel Restaurant                5
French Restaurant                11
German Restaurant                 1
Indian Restaurant                 4
Israeli Restaurant                1
Italian Restaurant               14
Japanese Restaurant               5
Jewish Restaurant                 1
Kebab Restaurant                  1
Korean Restaurant                 2
Lebanese Restaurant               2
Mediterranean Restaurant          3
Mexican Restaurant                1
Middle Eastern Restaurant         2
Modern European Restaurant        2
North

In [18]:
# let's store those info in a dataframe
# We first need to get all unique values 
rest_cat = [x for x in list(neighbor_df['cat'].unique())]
def getNbInDict(key,borough):
    if(key in freq[borough]): return freq[borough][key]
    else: return 0
freq_df = pd.DataFrame({'Borough' : boroughs})
for cat in rest_cat:
    freq_df[cat] = freq_df.apply(lambda x: getNbInDict(cat,x['Borough']),axis=1)
freq_df.head()

,Borough,Vietnamese Restaurant,Falafel Restaurant,French Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,German Restaurant,Sushi Restaurant,Modern European Restaurant,Chinese Restaurant,South American Restaurant,English Restaurant,American Restaurant,Ramen Restaurant,Indian Restaurant,Mediterranean Restaurant,Empanada Restaurant,Portuguese Restaurant,Turkish Restaurant,Spanish Restaurant,Japanese Restaurant,Szechuan Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Cantonese Restaurant,Tapas Restaurant,Israeli Restaurant,Lebanese Restaurant,Scandinavian Restaurant,North Indian Restaurant,Korean Restaurant,Eastern European Restaurant,Middle Eastern Restaurant,Kebab Restaurant,Argentinian Restaurant,Jewish Restaurant,Arepa Restaurant,Afghan Restaurant,Austrian Restaurant,Udon Restaurant,Caucasian Restaurant,Mexican Restaurant,Fast Food Restaurant,Greek Restaurant,Brazilian Restaurant,Thai Restaurant,Latin American Restaurant,Dim Sum Restaurant,Belgian Restaurant,Malay Restaurant,Okonomiyaki Restaurant,Peruvian Restaurant,Yoshoku Restaurant,Ethiopian Restaurant,Paella Restaurant,Persian Restaurant,New American Restaurant,Caribbean Restaurant,South Indian Restaurant,Sri Lankan Restaurant,Southern / Soul Food Restaurant,Polish Restaurant,Doner Restaurant,African Restaurant,Taiwanese Restaurant,Dumpling Restaurant,Yakitori Restaurant,Veneto Restaurant,Moroccan Restaurant,Ukrainian Restaurant,Bulgarian Restaurant,Hunan Restaurant,Shaanxi Restaurant,Indonesian Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,Australian Restaurant,Iraqi Restaurant,Pakistani Restaurant,Halal Restaurant,Scottish Restaurant,Russian Restaurant,Molecular Gastronomy Restaurant
0,City of London,10,5,11,5,14,17,1,5,2,1,2,4,4,1,4,3,1,2,2,1,5,1,1,3,1,5,1,2,1,1,2,1,2,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Barnet,0,0,1,0,5,0,0,3,0,2,0,0,1,0,3,3,0,0,3,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Brent,0,0,0,1,3,3,0,0,0,2,0,1,2,0,13,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bromley,0,0,0,1,2,0,0,2,0,0,0,1,1,0,3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
print(freq_df.shape)

(32, 84)


In [20]:
freq_df.to_csv('freq_restaurants.csv',index=False)

In [32]:
freq_df_ll = pd.merge(freq_df,geo_df.rename(columns={'Area_name':'Borough','latitude':'lat','longitude':'long'}),how='left',on='Borough')
freq_df_ll.head()

,Borough,Vietnamese Restaurant,Falafel Restaurant,French Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,German Restaurant,Sushi Restaurant,Modern European Restaurant,Chinese Restaurant,South American Restaurant,English Restaurant,American Restaurant,Ramen Restaurant,Indian Restaurant,Mediterranean Restaurant,Empanada Restaurant,Portuguese Restaurant,Turkish Restaurant,Spanish Restaurant,Japanese Restaurant,Szechuan Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Cantonese Restaurant,Tapas Restaurant,Israeli Restaurant,Lebanese Restaurant,Scandinavian Restaurant,North Indian Restaurant,Korean Restaurant,Eastern European Restaurant,Middle Eastern Restaurant,Kebab Restaurant,Argentinian Restaurant,Jewish Restaurant,Arepa Restaurant,Afghan Restaurant,Austrian Restaurant,Udon Restaurant,Caucasian Restaurant,Mexican Restaurant,Fast Food Restaurant,Greek Restaurant,Brazilian Restaurant,Thai Restaurant,Latin American Restaurant,Dim Sum Restaurant,Belgian Restaurant,Malay Restaurant,Okonomiyaki Restaurant,Peruvian Restaurant,Yoshoku Restaurant,Ethiopian Restaurant,Paella Restaurant,Persian Restaurant,New American Restaurant,Caribbean Restaurant,South Indian Restaurant,Sri Lankan Restaurant,Southern / Soul Food Restaurant,Polish Restaurant,Doner Restaurant,African Restaurant,Taiwanese Restaurant,Dumpling Restaurant,Yakitori Restaurant,Veneto Restaurant,Moroccan Restaurant,Ukrainian Restaurant,Bulgarian Restaurant,Hunan Restaurant,Shaanxi Restaurant,Indonesian Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,Australian Restaurant,Iraqi Restaurant,Pakistani Restaurant,Halal Restaurant,Scottish Restaurant,Russian Restaurant,Molecular Gastronomy Restaurant,lat,long
0,City of London,10,5,11,5,14,17,1,5,2,1,2,4,4,1,4,3,1,2,2,1,5,1,1,3,1,5,1,2,1,1,2,1,2,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.5155,-0.0922
1,Barking and Dagenham,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.5607,0.1557
2,Barnet,0,0,1,0,5,0,0,3,0,2,0,0,1,0,3,3,0,0,3,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.6252,-0.1517
3,Brent,0,0,0,1,3,3,0,0,0,2,0,1,2,0,13,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.5588,-0.2817
4,Bromley,0,0,0,1,2,0,0,2,0,0,0,1,1,0,3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.4039,0.0198


In [33]:
freq_df_ll['Nearby Venues Count'] = freq_df_ll.apply(lambda x: countNearbyVenues(x['lat'],x['long']),axis=1)
freq_df_ll.head()

,Borough,Vietnamese Restaurant,Falafel Restaurant,French Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,German Restaurant,Sushi Restaurant,Modern European Restaurant,Chinese Restaurant,South American Restaurant,English Restaurant,American Restaurant,Ramen Restaurant,Indian Restaurant,Mediterranean Restaurant,Empanada Restaurant,Portuguese Restaurant,Turkish Restaurant,Spanish Restaurant,Japanese Restaurant,Szechuan Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Cantonese Restaurant,Tapas Restaurant,Israeli Restaurant,Lebanese Restaurant,Scandinavian Restaurant,North Indian Restaurant,Korean Restaurant,Eastern European Restaurant,Middle Eastern Restaurant,Kebab Restaurant,Argentinian Restaurant,Jewish Restaurant,Arepa Restaurant,Afghan Restaurant,Austrian Restaurant,Udon Restaurant,Caucasian Restaurant,Mexican Restaurant,Fast Food Restaurant,Greek Restaurant,Brazilian Restaurant,Thai Restaurant,Latin American Restaurant,Dim Sum Restaurant,Belgian Restaurant,Malay Restaurant,Okonomiyaki Restaurant,Peruvian Restaurant,Yoshoku Restaurant,Ethiopian Restaurant,Paella Restaurant,Persian Restaurant,New American Restaurant,Caribbean Restaurant,South Indian Restaurant,Sri Lankan Restaurant,Southern / Soul Food Restaurant,Polish Restaurant,Doner Restaurant,African Restaurant,Taiwanese Restaurant,Dumpling Restaurant,Yakitori Restaurant,Veneto Restaurant,Moroccan Restaurant,Ukrainian Restaurant,Bulgarian Restaurant,Hunan Restaurant,Shaanxi Restaurant,Indonesian Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,Australian Restaurant,Iraqi Restaurant,Pakistani Restaurant,Halal Restaurant,Scottish Restaurant,Russian Restaurant,Molecular Gastronomy Restaurant,lat,long,Nearby Venues Count
0,City of London,10,5,11,5,14,17,1,5,2,1,2,4,4,1,4,3,1,2,2,1,5,1,1,3,1,5,1,2,1,1,2,1,2,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.5155,-0.0922,85
1,Barking and Dagenham,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.5607,0.1557,28
2,Barnet,0,0,1,0,5,0,0,3,0,2,0,0,1,0,3,3,0,0,3,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.6252,-0.1517,72
3,Brent,0,0,0,1,3,3,0,0,0,2,0,1,2,0,13,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.5588,-0.2817,79
4,Bromley,0,0,0,1,2,0,0,2,0,0,0,1,1,0,3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.4039,0.0198,35


In [34]:
freq_df_ll.to_csv('freq_ll_restaurants.csv',index=False)

In [2]:
freq_df_ll = pd.read_csv('freq_ll_restaurants.csv')
df_filtered = pd.read_csv('df_filtered.csv')

In [3]:
freq_df_ll.drop(['lat','long'],axis=1,inplace=True)

In [4]:
london_df = pd.merge(df_filtered.rename(columns={'Area_name':'Borough'}),freq_df_ll,how='left',on='Borough')

In [5]:
london_df = london_df.dropna()

In [6]:
london_df.head()

,GLA_Population_Estimate_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015","New_migrant_(NINo)_rates,_(2015/16)",Employment_rate_(%)_(2015),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),"Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)",%_children_living_in_out-of-work_households_(2015),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),People_aged_17+_with_diabetes_(%),latitude,longitude,Borough,Vietnamese Restaurant,Falafel Restaurant,French Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,German Restaurant,Sushi Restaurant,Modern European Restaurant,Chinese Restaurant,South American Restaurant,English Restaurant,American Restaurant,Ramen Restaurant,Indian Restaurant,Mediterranean Restaurant,Empanada Restaurant,Portuguese Restaurant,Turkish Restaurant,Spanish Restaurant,Japanese Restaurant,Szechuan Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Cantonese Restaurant,Tapas Restaurant,Israeli Restaurant,Lebanese Restaurant,Scandinavian Restaurant,North Indian Restaurant,Korean Restaurant,Eastern European Restaurant,Middle Eastern Restaurant,Kebab Restaurant,Argentinian Restaurant,Jewish Restaurant,Arepa Restaurant,Afghan Restaurant,Austrian Restaurant,Udon Restaurant,Caucasian Restaurant,Mexican Restaurant,Fast Food Restaurant,Greek Restaurant,Brazilian Restaurant,Thai Restaurant,Latin American Restaurant,Dim Sum Restaurant,Belgian Restaurant,Malay Restaurant,Okonomiyaki Restaurant,Peruvian Restaurant,Yoshoku Restaurant,Ethiopian Restaurant,Paella Restaurant,Persian Restaurant,New American Restaurant,Caribbean Restaurant,South Indian Restaurant,Sri Lankan Restaurant,Southern / Soul Food Restaurant,Polish Restaurant,Doner Restaurant,African Restaurant,Taiwanese Restaurant,Dumpling Restaurant,Yakitori Restaurant,Veneto Restaurant,Moroccan Restaurant,Ukrainian Restaurant,Bulgarian Restaurant,Hunan Restaurant,Shaanxi Restaurant,Indonesian Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,Australian Restaurant,Iraqi Restaurant,Pakistani Restaurant,Halal Restaurant,Scottish Restaurant,Russian Restaurant,Molecular Gastronomy Restaurant,Nearby Venues Count
0,8800,43.2,11.4,73.1,15.5,152.2,64.6,3.4,84.3,26130,64.3,1692,0.4,7.9,6.6,7.1,6.0,5.6,2.6,51.5155,-0.0922,City of London,10.0,5.0,11.0,5.0,14.0,17.0,1.0,5.0,2.0,1.0,2.0,4.0,4.0,1.0,4.0,3.0,1.0,2.0,2.0,1.0,5.0,1.0,1.0,3.0,1.0,5.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0
1,209000,32.9,27.2,63.1,9.7,59.1,65.8,21.1,0.5,6560,73.0,56966,0.8,18.7,7.1,7.6,7.1,3.1,7.3,51.5607,0.1557,Barking and Dagenham,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0
2,389600,37.3,21.1,64.9,14.0,53.1,68.5,18.7,0.7,26190,73.8,144717,1.1,9.3,7.5,7.8,7.4,2.8,6.0,51.6252,-0.1517,Barnet,0.0,0.0,1.0,0.0,5.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,0.0,3.0,3.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0
4,332100,35.6,20.9,67.8,11.3,100.9,69.5,17.6,0.6,15745,74.4,87802,0.8,13.7,7.3,7.4,7.2,2.9,7.9,51.5588,-0.2817,Brent,0.0,0.0,0.0,1.0,3.0,3.0,0.0,0.0

In [7]:
london_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 32
Columns: 106 entries, GLA_Population_Estimate_2017 to Nearby Venues Count
dtypes: float64(102), int64(3), object(1)
memory usage: 26.8+ KB


In [8]:
# Normalize data
non_object_column = [x for x in list(london_df.columns.values) if x!='Borough' and x!='latitude' and x!='longitude']
min_max_scaler = preprocessing.MinMaxScaler()
london_df[non_object_column] = min_max_scaler.fit_transform(london_df[non_object_column])
london_df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,GLA_Population_Estimate_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015","New_migrant_(NINo)_rates,_(2015/16)",Employment_rate_(%)_(2015),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),"Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)",%_children_living_in_out-of-work_households_(2015),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),People_aged_17+_with_diabetes_(%),latitude,longitude,Borough,Vietnamese Restaurant,Falafel Restaurant,French Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,German Restaurant,Sushi Restaurant,Modern European Restaurant,Chinese Restaurant,South American Restaurant,English Restaurant,American Restaurant,Ramen Restaurant,Indian Restaurant,Mediterranean Restaurant,Empanada Restaurant,Portuguese Restaurant,Turkish Restaurant,Spanish Restaurant,Japanese Restaurant,Szechuan Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Cantonese Restaurant,Tapas Restaurant,Israeli Restaurant,Lebanese Restaurant,Scandinavian Restaurant,North Indian Restaurant,Korean Restaurant,Eastern European Restaurant,Middle Eastern Restaurant,Kebab Restaurant,Argentinian Restaurant,Jewish Restaurant,Arepa Restaurant,Afghan Restaurant,Austrian Restaurant,Udon Restaurant,Caucasian Restaurant,Mexican Restaurant,Fast Food Restaurant,Greek Restaurant,Brazilian Restaurant,Thai Restaurant,Latin American Restaurant,Dim Sum Restaurant,Belgian Restaurant,Malay Restaurant,Okonomiyaki Restaurant,Peruvian Restaurant,Yoshoku Restaurant,Ethiopian Restaurant,Paella Restaurant,Persian Restaurant,New American Restaurant,Caribbean Restaurant,South Indian Restaurant,Sri Lankan Restaurant,Southern / Soul Food Restaurant,Polish Restaurant,Doner Restaurant,African Restaurant,Taiwanese Restaurant,Dumpling Restaurant,Yakitori Restaurant,Veneto Restaurant,Moroccan Restaurant,Ukrainian Restaurant,Bulgarian Restaurant,Hunan Restaurant,Shaanxi Restaurant,Indonesian Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,Australian Restaurant,Iraqi Restaurant,Pakistani Restaurant,Halal Restaurant,Scottish Restaurant,Russian Restaurant,Molecular Gastronomy Restaurant,Nearby Venues Count
0,0.000000,1.000000,0.000000,0.830769,0.766129,1.000000,0.000000,0.000000,1.000000,0.400819,0.033333,0.000000,0.000,0.124294,0.0,0.000,0.0000,1.000000,0.000000,51.5155,-0.0922,City of London,1.0,1.0,1.000000,0.714286,0.666667,1.000000,0.5,1.0,0.666667,0.125,1.0,0.8,1.00,0.25,0.266667,0.6,1.0,1.0,0.333333,0.333333,0.454545,0.5,0.2,0.6,1.0,0.833333,1.0,0.4,0.5,1.0,0.25,0.5,0.181818,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
1,0.525735,0.127119,1.000000,0.061538,0.298387,0.324383,0.080000,0.740586,0.001192,0.000000,0.613333,0.363129,0.500,0.734463,0.5,0.625,0.6875,0.166667,0.796610,51.5607,0.1557,Barking and Dagenham,0.0,0.0,0.000000,0.000000,0.000000,0.058824,0.0,0.0,0.000000,0.125,0.0,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,1.000000,0.500000,0.613924,0.200000,0.645161,0.280842,0.260000,0.640167,0.003576,0.402048,0.666667,0.939619,0.875,0.203390,0.9,0.875,0.8750,0.066667,0.576271,51.6252,-0.1517,Barnet,0.0,0.0,0.090909,0.000000,0.238095,0.000000,0.0,0.6,0.000000,0.250,0.0,0.0,0.25,0.00,0.200000,0.6,0.0,0.0,0.500000,0.333333,0.000000,0.0,0

In [9]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_df.drop(['Borough','latitude','longitude'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 1, 3, 1, 1, 1, 1], dtype=int32)

In [10]:
# add clustering labels
london_grouped_clustering.insert(0, 'Cluster Labels', kmeans.labels_)
london_grouped_clustering.insert(0, 'Borough', london_df['Borough'])
london_grouped_clustering.insert(0, 'latitude', london_df['latitude'])
london_grouped_clustering.insert(0, 'longitude', london_df['longitude'])
london_grouped_clustering.head()

,longitude,latitude,Borough,Cluster Labels,GLA_Population_Estimate_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015","New_migrant_(NINo)_rates,_(2015/16)",Employment_rate_(%)_(2015),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),"Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)",%_children_living_in_out-of-work_households_(2015),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),People_aged_17+_with_diabetes_(%),Vietnamese Restaurant,Falafel Restaurant,French Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,German Restaurant,Sushi Restaurant,Modern European Restaurant,Chinese Restaurant,South American Restaurant,English Restaurant,American Restaurant,Ramen Restaurant,Indian Restaurant,Mediterranean Restaurant,Empanada Restaurant,Portuguese Restaurant,Turkish Restaurant,Spanish Restaurant,Japanese Restaurant,Szechuan Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Cantonese Restaurant,Tapas Restaurant,Israeli Restaurant,Lebanese Restaurant,Scandinavian Restaurant,North Indian Restaurant,Korean Restaurant,Eastern European Restaurant,Middle Eastern Restaurant,Kebab Restaurant,Argentinian Restaurant,Jewish Restaurant,Arepa Restaurant,Afghan Restaurant,Austrian Restaurant,Udon Restaurant,Caucasian Restaurant,Mexican Restaurant,Fast Food Restaurant,Greek Restaurant,Brazilian Restaurant,Thai Restaurant,Latin American Restaurant,Dim Sum Restaurant,Belgian Restaurant,Malay Restaurant,Okonomiyaki Restaurant,Peruvian Restaurant,Yoshoku Restaurant,Ethiopian Restaurant,Paella Restaurant,Persian Restaurant,New American Restaurant,Caribbean Restaurant,South Indian Restaurant,Sri Lankan Restaurant,Southern / Soul Food Restaurant,Polish Restaurant,Doner Restaurant,African Restaurant,Taiwanese Restaurant,Dumpling Restaurant,Yakitori Restaurant,Veneto Restaurant,Moroccan Restaurant,Ukrainian Restaurant,Bulgarian Restaurant,Hunan Restaurant,Shaanxi Restaurant,Indonesian Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,Australian Restaurant,Iraqi Restaurant,Pakistani Restaurant,Halal Restaurant,Scottish Restaurant,Russian Restaurant,Molecular Gastronomy Restaurant,Nearby Venues Count
0,-0.0922,51.5155,City of London,0,0.000000,1.000000,0.000000,0.830769,0.766129,1.000000,0.000000,0.000000,1.000000,0.400819,0.033333,0.000000,0.000,0.124294,0.0,0.000,0.0000,1.000000,0.000000,1.0,1.0,1.000000,0.714286,0.666667,1.000000,0.5,1.0,0.666667,0.125,1.0,0.8,1.00,0.25,0.266667,0.6,1.0,1.0,0.333333,0.333333,0.454545,0.5,0.2,0.6,1.0,0.833333,1.0,0.4,0.5,1.0,0.25,0.5,0.181818,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
1,0.1557,51.5607,Barking and Dagenham,1,0.525735,0.127119,1.000000,0.061538,0.298387,0.324383,0.080000,0.740586,0.001192,0.000000,0.613333,0.363129,0.500,0.734463,0.5,0.625,0.6875,0.166667,0.796610,0.0,0.0,0.000000,0.000000,0.000000,0.058824,0.0,0.0,0.000000,0.125,0.0,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,-0.1517,51.6252,Barnet,1,1.000000,0.500000,0.613924,0.200000,0.645161,0.280842,0.260000,0.640167,0.003576,0.402048,0.666667,0.939619,0.875,0.203390,0.9,0.875,0.8750,0.066667,0.576271,0.0,0.0,0.090909,0.000000,0.238095,0.000000,0.0,0.6,0.000000,0.250,0.0,0.0,0.25,0.00,0.200000,0.6,0.0,0.0,0.500000,0.

### Let's visualize first the borough position

In [11]:
# create map of London using latitude and longitude values
latitude = 51.5073219
longitude = -0.1276474
map_l = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(london_df['latitude'], london_df['longitude'], london_df['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_l)  
    
map_l

### Now let's see the five clusters

In [12]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['latitude'], 
                                  london_grouped_clustering['longitude'], 
                                  london_grouped_clustering['Borough'], 
                                  london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [13]:
cl1 = london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[2] + list(range(4, london_grouped_clustering.shape[1]))]]
cl1

,Borough,GLA_Population_Estimate_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015","New_migrant_(NINo)_rates,_(2015/16)",Employment_rate_(%)_(2015),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),"Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)",%_children_living_in_out-of-work_households_(2015),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),People_aged_17+_with_diabetes_(%),Vietnamese Restaurant,Falafel Restaurant,French Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,German Restaurant,Sushi Restaurant,Modern European Restaurant,Chinese Restaurant,South American Restaurant,English Restaurant,American Restaurant,Ramen Restaurant,Indian Restaurant,Mediterranean Restaurant,Empanada Restaurant,Portuguese Restaurant,Turkish Restaurant,Spanish Restaurant,Japanese Restaurant,Szechuan Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Cantonese Restaurant,Tapas Restaurant,Israeli Restaurant,Lebanese Restaurant,Scandinavian Restaurant,North Indian Restaurant,Korean Restaurant,Eastern European Restaurant,Middle Eastern Restaurant,Kebab Restaurant,Argentinian Restaurant,Jewish Restaurant,Arepa Restaurant,Afghan Restaurant,Austrian Restaurant,Udon Restaurant,Caucasian Restaurant,Mexican Restaurant,Fast Food Restaurant,Greek Restaurant,Brazilian Restaurant,Thai Restaurant,Latin American Restaurant,Dim Sum Restaurant,Belgian Restaurant,Malay Restaurant,Okonomiyaki Restaurant,Peruvian Restaurant,Yoshoku Restaurant,Ethiopian Restaurant,Paella Restaurant,Persian Restaurant,New American Restaurant,Caribbean Restaurant,South Indian Restaurant,Sri Lankan Restaurant,Southern / Soul Food Restaurant,Polish Restaurant,Doner Restaurant,African Restaurant,Taiwanese Restaurant,Dumpling Restaurant,Yakitori Restaurant,Veneto Restaurant,Moroccan Restaurant,Ukrainian Restaurant,Bulgarian Restaurant,Hunan Restaurant,Shaanxi Restaurant,Indonesian Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,Australian Restaurant,Iraqi Restaurant,Pakistani Restaurant,Halal Restaurant,Scottish Restaurant,Russian Restaurant,Molecular Gastronomy Restaurant,Nearby Venues Count
0,City of London,0.0,1.0,0.0,0.830769,0.766129,1.0,0.0,0.0,1.0,0.400819,0.033333,0.0,0.0,0.124294,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.714286,0.666667,1.0,0.5,1.0,0.666667,0.125,1.0,0.8,1.0,0.25,0.266667,0.6,1.0,1.0,0.333333,0.333333,0.454545,0.5,0.2,0.6,1.0,0.833333,1.0,0.4,0.5,1.0,0.25,0.5,0.181818,0.5,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
# As we can see this cluster is unique because it is indeed not a borough.

In [15]:
restaurants = [x for x in list(london_grouped_clustering.columns.values) if 'Restaurant' in x]
print(restaurants)

['Vietnamese Restaurant', 'Falafel Restaurant', 'French Restaurant', 'Seafood Restaurant', 'Italian Restaurant', 'Restaurant', 'German Restaurant', 'Sushi Restaurant', 'Modern European Restaurant', 'Chinese Restaurant', 'South American Restaurant', 'English Restaurant', 'American Restaurant', 'Ramen Restaurant', 'Indian Restaurant', 'Mediterranean Restaurant', 'Empanada Restaurant', 'Portuguese Restaurant', 'Turkish Restaurant', 'Spanish Restaurant', 'Japanese Restaurant', 'Szechuan Restaurant', 'Vegetarian / Vegan Restaurant', 'Asian Restaurant', 'Cantonese Restaurant', 'Tapas Restaurant', 'Israeli Restaurant', 'Lebanese Restaurant', 'Scandinavian Restaurant', 'North Indian Restaurant', 'Korean Restaurant', 'Eastern European Restaurant', 'Middle Eastern Restaurant', 'Kebab Restaurant', 'Argentinian Restaurant', 'Jewish Restaurant', 'Arepa Restaurant', 'Afghan Restaurant', 'Austrian Restaurant', 'Udon Restaurant', 'Caucasian Restaurant', 'Mexican Restaurant', 'Fast Food Restaurant', 'G

In [16]:
# Most frequent restaurants in this cluster
cl1_rest = cl1[restaurants].sum()[cl1[restaurants].sum()>0].sort_values(ascending=False)
cl1_rest.head()

Vietnamese Restaurant    1.0
French Restaurant        1.0
American Restaurant      1.0
Sushi Restaurant         1.0
Empanada Restaurant      1.0
dtype: float64

In [17]:
# Economic situation of cluster 1
cl1_eco = cl1[[x for x in list(london_grouped_clustering.columns.values) if x not in restaurants and x!='Cluster Labels' and x!='longitude' and x!='latitude']].mean(axis=0)
cl1_eco

GLA_Population_Estimate_2017                          0.000000
Average_Age,_2017                                     1.000000
Proportion_of_population_aged_0-15,_2015              0.000000
Proportion_of_population_of_working-age,_2015         0.830769
Proportion_of_population_aged_65_and_over,_2015       0.766129
New_migrant_(NINo)_rates,_(2015/16)                   1.000000
Employment_rate_(%)_(2015)                            0.000000
%_of_employment_that_is_in_public_sector_(2014)       0.000000
Jobs_Density,_2015                                    1.000000
Number_of_active_businesses,_2015                     0.400819
Two-year_business_survival_rates_(started_in_2013)    0.033333
Number_of_cars,_(2011_Census)                         0.000000
Number_of_cars_per_household,_(2011_Census)           0.000000
%_children_living_in_out-of-work_households_(2015)    0.124294
Life_satisfaction_score_2011-14_(out_of_10)           0.000000
Worthwhileness_score_2011-14_(out_of_10)              0

### Cluster 2

In [18]:
cl2 = london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[2] + list(range(4, london_grouped_clustering.shape[1]))]]
cl2

,Borough,GLA_Population_Estimate_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015","New_migrant_(NINo)_rates,_(2015/16)",Employment_rate_(%)_(2015),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),"Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)",%_children_living_in_out-of-work_households_(2015),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),People_aged_17+_with_diabetes_(%),Vietnamese Restaurant,Falafel Restaurant,French Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,German Restaurant,Sushi Restaurant,Modern European Restaurant,Chinese Restaurant,South American Restaurant,English Restaurant,American Restaurant,Ramen Restaurant,Indian Restaurant,Mediterranean Restaurant,Empanada Restaurant,Portuguese Restaurant,Turkish Restaurant,Spanish Restaurant,Japanese Restaurant,Szechuan Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Cantonese Restaurant,Tapas Restaurant,Israeli Restaurant,Lebanese Restaurant,Scandinavian Restaurant,North Indian Restaurant,Korean Restaurant,Eastern European Restaurant,Middle Eastern Restaurant,Kebab Restaurant,Argentinian Restaurant,Jewish Restaurant,Arepa Restaurant,Afghan Restaurant,Austrian Restaurant,Udon Restaurant,Caucasian Restaurant,Mexican Restaurant,Fast Food Restaurant,Greek Restaurant,Brazilian Restaurant,Thai Restaurant,Latin American Restaurant,Dim Sum Restaurant,Belgian Restaurant,Malay Restaurant,Okonomiyaki Restaurant,Peruvian Restaurant,Yoshoku Restaurant,Ethiopian Restaurant,Paella Restaurant,Persian Restaurant,New American Restaurant,Caribbean Restaurant,South Indian Restaurant,Sri Lankan Restaurant,Southern / Soul Food Restaurant,Polish Restaurant,Doner Restaurant,African Restaurant,Taiwanese Restaurant,Dumpling Restaurant,Yakitori Restaurant,Veneto Restaurant,Moroccan Restaurant,Ukrainian Restaurant,Bulgarian Restaurant,Hunan Restaurant,Shaanxi Restaurant,Indonesian Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,Australian Restaurant,Iraqi Restaurant,Pakistani Restaurant,Halal Restaurant,Scottish Restaurant,Russian Restaurant,Molecular Gastronomy Restaurant,Nearby Venues Count
1,Barking and Dagenham,0.525735,0.127119,1.000000,0.061538,0.298387,0.324383,0.080000,0.740586,0.001192,0.000000,0.613333,0.363129,0.500,0.734463,0.5,0.625,0.6875,0.166667,0.796610,0.0,0.0,0.000000,0.000000,0.000000,0.058824,0.0,0.0,0.0,0.125,0.0,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Barnet,1.000000,0.500000,0.613924,0.200000,0.645161,0.280842,0.260000,0.640167,0.003576,0.402048,0.666667,0.939619,0.875,0.203390,0.9,0.875,0.8750,0.066667,0.576271,0.0,0.0,0.090909,0.000000,0.238095,0.000000,0.0,0.6,0.0,0.250,0.0,0.0,0.25,0.00,0.200000,0.6,0.0,0.0,0.500000,0.333333,0.000000,0.0,0.0,0.2,0.0,0.000000,0.0,0.2,0.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.2,0.2,0.5,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.771930
4,Brent,0.849002,0.355932,0.601266,0.423077,0.427419,0.627721,0.326667,0.594142,0.002384,0.188121,0.706667,0.565709,0.500,0.451977,0.7,0.375,0.7500,0.100000,0.898305,0.0,0.0,0.000000,0.142857,0.142857,0.176471,0.0,0.0,0.0,0.250,0.0,0.2,0.50,0.00,0.866667,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,1.0,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0

In [19]:
# Most frequent restaurants in this cluster
cl2_rest = cl2[restaurants].sum()[cl2[restaurants].sum()>0].sort_values(ascending=False)
cl2_rest.head()

Indian Restaurant       5.666667
Fast Food Restaurant    5.600000
Chinese Restaurant      4.125000
Brazilian Restaurant    3.000000
Malay Restaurant        3.000000
dtype: float64

In [20]:
# Economic situation of cluster 2
cl2_eco = cl2[[x for x in list(london_grouped_clustering.columns.values) if x not in restaurants and x!='Cluster Labels' and x!='longitude' and x!='latitude']].mean(axis=0)
cl2_eco

GLA_Population_Estimate_2017                          0.718400
Average_Age,_2017                                     0.463277
Proportion_of_population_aged_0-15,_2015              0.630098
Proportion_of_population_of_working-age,_2015         0.252564
Proportion_of_population_aged_65_and_over,_2015       0.569444
New_migrant_(NINo)_rates,_(2015/16)                   0.215086
Employment_rate_(%)_(2015)                            0.601852
%_of_employment_that_is_in_public_sector_(2014)       0.600186
Jobs_Density,_2015                                    0.003046
Number_of_active_businesses,_2015                     0.143318
Two-year_business_survival_rates_(started_in_2013)    0.779630
Number_of_cars,_(2011_Census)                         0.657102
Number_of_cars_per_household,_(2011_Census)           0.770833
%_children_living_in_out-of-work_households_(2015)    0.375392
Life_satisfaction_score_2011-14_(out_of_10)           0.722222
Worthwhileness_score_2011-14_(out_of_10)              0

### Cluster 3

In [21]:
cl3 = london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[2] + list(range(4, london_grouped_clustering.shape[1]))]]
cl3

,Borough,GLA_Population_Estimate_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015","New_migrant_(NINo)_rates,_(2015/16)",Employment_rate_(%)_(2015),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),"Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)",%_children_living_in_out-of-work_households_(2015),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),People_aged_17+_with_diabetes_(%),Vietnamese Restaurant,Falafel Restaurant,French Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,German Restaurant,Sushi Restaurant,Modern European Restaurant,Chinese Restaurant,South American Restaurant,English Restaurant,American Restaurant,Ramen Restaurant,Indian Restaurant,Mediterranean Restaurant,Empanada Restaurant,Portuguese Restaurant,Turkish Restaurant,Spanish Restaurant,Japanese Restaurant,Szechuan Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Cantonese Restaurant,Tapas Restaurant,Israeli Restaurant,Lebanese Restaurant,Scandinavian Restaurant,North Indian Restaurant,Korean Restaurant,Eastern European Restaurant,Middle Eastern Restaurant,Kebab Restaurant,Argentinian Restaurant,Jewish Restaurant,Arepa Restaurant,Afghan Restaurant,Austrian Restaurant,Udon Restaurant,Caucasian Restaurant,Mexican Restaurant,Fast Food Restaurant,Greek Restaurant,Brazilian Restaurant,Thai Restaurant,Latin American Restaurant,Dim Sum Restaurant,Belgian Restaurant,Malay Restaurant,Okonomiyaki Restaurant,Peruvian Restaurant,Yoshoku Restaurant,Ethiopian Restaurant,Paella Restaurant,Persian Restaurant,New American Restaurant,Caribbean Restaurant,South Indian Restaurant,Sri Lankan Restaurant,Southern / Soul Food Restaurant,Polish Restaurant,Doner Restaurant,African Restaurant,Taiwanese Restaurant,Dumpling Restaurant,Yakitori Restaurant,Veneto Restaurant,Moroccan Restaurant,Ukrainian Restaurant,Bulgarian Restaurant,Hunan Restaurant,Shaanxi Restaurant,Indonesian Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,Australian Restaurant,Iraqi Restaurant,Pakistani Restaurant,Halal Restaurant,Scottish Restaurant,Russian Restaurant,Molecular Gastronomy Restaurant,Nearby Venues Count
11,Hackney,0.697216,0.144068,0.588608,0.753846,0.096774,0.229318,0.293333,0.615063,0.003576,0.244752,0.866667,0.263494,0.000,0.790960,0.4,0.250,0.6250,0.400000,0.542373,0.2,0.0,0.090909,0.142857,0.142857,0.411765,0.0,0.0,0.333333,0.000,0.0,0.0,0.00,0.25,0.066667,0.0,0.0,0.0,0.500000,0.333333,0.090909,0.0,0.8,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.181818,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
12,Hammersmith and Fulham,0.463498,0.364407,0.379747,0.769231,0.346774,0.413643,0.860000,0.543933,0.009535,0.154327,0.640000,0.276916,0.125,0.587571,0.8,0.500,0.7500,0.166667,0.305085,0.2,0.4,0.363636,0.000000,0.428571,0.117647,0.0,0.2,0.333333,0.500,0.0,0.0,0.00,0.00,0.533333,0.2,0.0,0.0,0.333333,0.000000,0.545455,0.0,0.2,0.0,0.0,0.333333,0.0,0.2,0.0,0.0,0.000,0.0,0.727273,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.6,0.2,0.0,1.000000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.754386
13,Haringey,0.706933,0.313559,0.544304,0.646154,0.266129,0.465167,0.446667,0.602510,0.001192,0.125243,0.706667,0.393014,0.250,0.632768,0.6,0.500,0.7500,0.200000,0.559322,0.0,0.0,0.000000,0.000000,0.095238,0.117647,0.0,0.0,0.000000,0.000,0.0,0.0,0.00,0.00,0.400000,0.8,0.0,0.0,0.666667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.000,0.0,0.272727

In [22]:
# Most frequent restaurants in this cluster
cl3_rest = cl3[restaurants].sum()[cl3[restaurants].sum()>0].sort_values(ascending=False)
cl3_rest.head()

Turkish Restaurant               4.166667
Mediterranean Restaurant         3.200000
Modern European Restaurant       3.000000
Vegetarian / Vegan Restaurant    3.000000
Thai Restaurant                  2.888889
dtype: float64

In [23]:
# Economic situation of cluster 3
cl3_eco = cl3[[x for x in list(london_grouped_clustering.columns.values) if x not in restaurants and x!='Cluster Labels' and x!='longitude' and x!='latitude']].mean(axis=0)
cl3_eco

GLA_Population_Estimate_2017                          0.726917
Average_Age,_2017                                     0.230508
Proportion_of_population_aged_0-15,_2015              0.497468
Proportion_of_population_of_working-age,_2015         0.771538
Proportion_of_population_aged_65_and_over,_2015       0.192742
New_migrant_(NINo)_rates,_(2015/16)                   0.380044
Employment_rate_(%)_(2015)                            0.570667
%_of_employment_that_is_in_public_sector_(2014)       0.662343
Jobs_Density,_2015                                    0.005959
Number_of_active_businesses,_2015                     0.195289
Two-year_business_survival_rates_(started_in_2013)    0.552000
Number_of_cars,_(2011_Census)                         0.372173
Number_of_cars_per_household,_(2011_Census)           0.175000
%_children_living_in_out-of-work_households_(2015)    0.688136
Life_satisfaction_score_2011-14_(out_of_10)           0.630000
Worthwhileness_score_2011-14_(out_of_10)              0

### Cluster 4

In [24]:
cl4 = london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[2] + list(range(4, london_grouped_clustering.shape[1]))]]
cl4

,Borough,GLA_Population_Estimate_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015","New_migrant_(NINo)_rates,_(2015/16)",Employment_rate_(%)_(2015),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),"Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)",%_children_living_in_out-of-work_households_(2015),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),People_aged_17+_with_diabetes_(%),Vietnamese Restaurant,Falafel Restaurant,French Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,German Restaurant,Sushi Restaurant,Modern European Restaurant,Chinese Restaurant,South American Restaurant,English Restaurant,American Restaurant,Ramen Restaurant,Indian Restaurant,Mediterranean Restaurant,Empanada Restaurant,Portuguese Restaurant,Turkish Restaurant,Spanish Restaurant,Japanese Restaurant,Szechuan Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Cantonese Restaurant,Tapas Restaurant,Israeli Restaurant,Lebanese Restaurant,Scandinavian Restaurant,North Indian Restaurant,Korean Restaurant,Eastern European Restaurant,Middle Eastern Restaurant,Kebab Restaurant,Argentinian Restaurant,Jewish Restaurant,Arepa Restaurant,Afghan Restaurant,Austrian Restaurant,Udon Restaurant,Caucasian Restaurant,Mexican Restaurant,Fast Food Restaurant,Greek Restaurant,Brazilian Restaurant,Thai Restaurant,Latin American Restaurant,Dim Sum Restaurant,Belgian Restaurant,Malay Restaurant,Okonomiyaki Restaurant,Peruvian Restaurant,Yoshoku Restaurant,Ethiopian Restaurant,Paella Restaurant,Persian Restaurant,New American Restaurant,Caribbean Restaurant,South Indian Restaurant,Sri Lankan Restaurant,Southern / Soul Food Restaurant,Polish Restaurant,Doner Restaurant,African Restaurant,Taiwanese Restaurant,Dumpling Restaurant,Yakitori Restaurant,Veneto Restaurant,Moroccan Restaurant,Ukrainian Restaurant,Bulgarian Restaurant,Hunan Restaurant,Shaanxi Restaurant,Indonesian Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,Australian Restaurant,Iraqi Restaurant,Pakistani Restaurant,Halal Restaurant,Scottish Restaurant,Russian Restaurant,Molecular Gastronomy Restaurant,Nearby Venues Count
6,Camden,0.613708,0.423729,0.373418,0.669231,0.459677,0.335994,0.306667,0.376569,0.021454,0.508449,0.653333,0.295035,0.125,0.610169,0.5,0.375,0.6875,0.333333,0.220339,0.3,0.2,0.727273,0.428571,0.714286,0.529412,0.0,0.8,1.000000,0.375,0.5,1.0,0.25,1.00,0.466667,0.8,0.0,0.5,1.000000,0.666667,1.000000,0.0,1.0,1.0,0.0,0.666667,0.0,0.2,1.0,0.0,1.0,0.0,0.454545,1.0,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.666667,0.2,1.0,0.5,0.444444,0.0,0.0,1.0,1.0,1.0,0.666667,1.0,1.0,1.0,0.2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.964912
19,Kensington and Chelsea,0.394433,0.669492,0.316456,0.538462,0.669355,0.375907,0.240000,0.472803,0.011919,0.159549,0.713333,0.281468,0.250,0.299435,1.0,0.875,1.0000,0.166667,0.271186,0.0,0.0,0.454545,0.142857,1.000000,0.529412,0.0,0.2,0.666667,1.000,0.0,0.4,0.25,0.25,0.600000,0.6,0.0,0.0,0.333333,0.000000,0.545455,0.0,0.0,0.2,0.0,0.666667,0.0,1.0,0.5,0.0,0.0,0.0,1.000000,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.2,1.0,0.0,0.555556,0.0,0.5,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.701754


In [25]:
# Most frequent restaurants in this cluster
cl4_rest = cl4[restaurants].sum()[cl4[restaurants].sum()>0].sort_values(ascending=False)
cl4_rest.head()

Kebab Restaurant              2.000000
Malay Restaurant              2.000000
Greek Restaurant              2.000000
Italian Restaurant            1.714286
Modern European Restaurant    1.666667
dtype: float64

In [26]:
# Economic situation of cluster 4
cl4_eco = cl4[[x for x in list(london_grouped_clustering.columns.values) if x not in restaurants and x!='Cluster Labels' and x!='longitude' and x!='latitude']].mean(axis=0)
cl4_eco

GLA_Population_Estimate_2017                          0.504070
Average_Age,_2017                                     0.546610
Proportion_of_population_aged_0-15,_2015              0.344937
Proportion_of_population_of_working-age,_2015         0.603846
Proportion_of_population_aged_65_and_over,_2015       0.564516
New_migrant_(NINo)_rates,_(2015/16)                   0.355951
Employment_rate_(%)_(2015)                            0.273333
%_of_employment_that_is_in_public_sector_(2014)       0.424686
Jobs_Density,_2015                                    0.016687
Number_of_active_businesses,_2015                     0.333999
Two-year_business_survival_rates_(started_in_2013)    0.683333
Number_of_cars,_(2011_Census)                         0.288252
Number_of_cars_per_household,_(2011_Census)           0.187500
%_children_living_in_out-of-work_households_(2015)    0.454802
Life_satisfaction_score_2011-14_(out_of_10)           0.750000
Worthwhileness_score_2011-14_(out_of_10)              0

### Cluster 5

In [27]:
cl5 = london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[2] + list(range(4, london_grouped_clustering.shape[1]))]]
cl5

,Borough,GLA_Population_Estimate_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015","New_migrant_(NINo)_rates,_(2015/16)",Employment_rate_(%)_(2015),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),"Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)",%_children_living_in_out-of-work_households_(2015),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),People_aged_17+_with_diabetes_(%),Vietnamese Restaurant,Falafel Restaurant,French Restaurant,Seafood Restaurant,Italian Restaurant,Restaurant,German Restaurant,Sushi Restaurant,Modern European Restaurant,Chinese Restaurant,South American Restaurant,English Restaurant,American Restaurant,Ramen Restaurant,Indian Restaurant,Mediterranean Restaurant,Empanada Restaurant,Portuguese Restaurant,Turkish Restaurant,Spanish Restaurant,Japanese Restaurant,Szechuan Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant,Cantonese Restaurant,Tapas Restaurant,Israeli Restaurant,Lebanese Restaurant,Scandinavian Restaurant,North Indian Restaurant,Korean Restaurant,Eastern European Restaurant,Middle Eastern Restaurant,Kebab Restaurant,Argentinian Restaurant,Jewish Restaurant,Arepa Restaurant,Afghan Restaurant,Austrian Restaurant,Udon Restaurant,Caucasian Restaurant,Mexican Restaurant,Fast Food Restaurant,Greek Restaurant,Brazilian Restaurant,Thai Restaurant,Latin American Restaurant,Dim Sum Restaurant,Belgian Restaurant,Malay Restaurant,Okonomiyaki Restaurant,Peruvian Restaurant,Yoshoku Restaurant,Ethiopian Restaurant,Paella Restaurant,Persian Restaurant,New American Restaurant,Caribbean Restaurant,South Indian Restaurant,Sri Lankan Restaurant,Southern / Soul Food Restaurant,Polish Restaurant,Doner Restaurant,African Restaurant,Taiwanese Restaurant,Dumpling Restaurant,Yakitori Restaurant,Veneto Restaurant,Moroccan Restaurant,Ukrainian Restaurant,Bulgarian Restaurant,Hunan Restaurant,Shaanxi Restaurant,Indonesian Restaurant,Cajun / Creole Restaurant,Filipino Restaurant,Australian Restaurant,Iraqi Restaurant,Pakistani Restaurant,Halal Restaurant,Scottish Restaurant,Russian Restaurant,Molecular Gastronomy Restaurant,Nearby Venues Count
32,Westminster,0.612658,0.533898,0.28481,0.769231,0.459677,0.417997,0.066667,0.41841,0.046484,1.0,0.333333,0.309547,0.125,0.570621,0.7,0.5,0.6875,0.266667,0.305085,0.0,0.0,1.0,1.0,0.52381,0.764706,0.0,0.2,0.666667,0.375,0.0,0.4,0.75,0.0,0.6,0.6,0.0,0.0,0.0,0.333333,0.454545,0.0,0.2,0.4,1.0,0.333333,1.0,0.0,0.5,1.0,0.375,0.0,0.090909,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.111111,1.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0


In [28]:
# Most frequent restaurants in this cluster
cl5_rest = cl5[restaurants].sum()[cl5[restaurants].sum()>0].sort_values(ascending=False)
cl5_rest.head()

Molecular Gastronomy Restaurant    1.0
Latin American Restaurant          1.0
Seafood Restaurant                 1.0
Russian Restaurant                 1.0
Israeli Restaurant                 1.0
dtype: float64

In [29]:
# Economic situation of cluster 5
cl5_eco = cl5[[x for x in list(london_grouped_clustering.columns.values) if x not in restaurants and x!='Cluster Labels' and x!='longitude' and x!='latitude']].mean(axis=0)
cl5_eco

GLA_Population_Estimate_2017                          0.612658
Average_Age,_2017                                     0.533898
Proportion_of_population_aged_0-15,_2015              0.284810
Proportion_of_population_of_working-age,_2015         0.769231
Proportion_of_population_aged_65_and_over,_2015       0.459677
New_migrant_(NINo)_rates,_(2015/16)                   0.417997
Employment_rate_(%)_(2015)                            0.066667
%_of_employment_that_is_in_public_sector_(2014)       0.418410
Jobs_Density,_2015                                    0.046484
Number_of_active_businesses,_2015                     1.000000
Two-year_business_survival_rates_(started_in_2013)    0.333333
Number_of_cars,_(2011_Census)                         0.309547
Number_of_cars_per_household,_(2011_Census)           0.125000
%_children_living_in_out-of-work_households_(2015)    0.570621
Life_satisfaction_score_2011-14_(out_of_10)           0.700000
Worthwhileness_score_2011-14_(out_of_10)              0

In [30]:
df_eco = pd.DataFrame(columns = list(cl1_eco.index.values))
df_eco = pd.concat([df_eco,pd.DataFrame(cl1_eco).T])
df_eco = pd.concat([df_eco,pd.DataFrame(cl2_eco).T])
df_eco = pd.concat([df_eco,pd.DataFrame(cl3_eco).T])
df_eco = pd.concat([df_eco,pd.DataFrame(cl4_eco).T])
df_eco = pd.concat([df_eco,pd.DataFrame(cl5_eco).T])
df_eco.head()

,GLA_Population_Estimate_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015","New_migrant_(NINo)_rates,_(2015/16)",Employment_rate_(%)_(2015),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),"Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)",%_children_living_in_out-of-work_households_(2015),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),People_aged_17+_with_diabetes_(%),Nearby Venues Count
0,0.000000,1.000000,0.000000,0.830769,0.766129,1.000000,0.000000,0.000000,1.000000,0.400819,0.033333,0.000000,0.000000,0.124294,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
0,0.718400,0.463277,0.630098,0.252564,0.569444,0.215086,0.601852,0.600186,0.003046,0.143318,0.779630,0.657102,0.770833,0.375392,0.722222,0.666667,0.798611,0.203704,0.635593,0.622807
0,0.726917,0.230508,0.497468,0.771538,0.192742,0.380044,0.570667,0.662343,0.005959,0.195289,0.552000,0.372173,0.175000,0.688136,0.630000,0.475000,0.743750,0.270000,0.515254,0.900000
0,0.504070,0.546610,0.344937,0.603846,0.564516,0.355951,0.273333,0.424686,0.016687,0.333999,0.683333,0.288252,0.187500,0.454802,0.750000,0.625000,0.843750,0.250000,0.245763,0.833333
0,0.612658,0.533898,0.284810,0.769231,0.459677,0.417997,0.066667,0.418410,0.046484,1.000000,0.333333,0.309547,0.125000,0.570621,0.700000,0.500000,0.687500,0.266667,0.305085,1.000000


In [31]:
df_rest = pd.DataFrame(columns = restaurants)
df_rest = pd.concat([df_rest,pd.DataFrame(cl1_rest).T.set_index(pd.Index([0]))],sort=True).fillna(0)
df_rest = pd.concat([df_rest,pd.DataFrame(cl2_rest).T.set_index(pd.Index([1]))],sort=True).fillna(0)
df_rest = pd.concat([df_rest,pd.DataFrame(cl3_rest).T.set_index(pd.Index([2]))],sort=True).fillna(0)
df_rest = pd.concat([df_rest,pd.DataFrame(cl4_rest).T.set_index(pd.Index([3]))],sort=True).fillna(0)
df_rest = pd.concat([df_rest,pd.DataFrame(cl5_rest).T.set_index(pd.Index([4]))],sort=True).fillna(0)
df_rest.head()

,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Belgian Restaurant,Brazilian Restaurant,Bulgarian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Hunan Restaurant,Indian Restaurant,Indonesian Restaurant,Iraqi Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Okonomiyaki Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Scandinavian Restaurant,Scottish Restaurant,Seafood Restaurant,Shaanxi Restaurant,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Veneto Restaurant,Vietnamese Restaurant,Yakitori Restaurant,Yoshoku Restaurant
0,1.0,0.0,1.00,1.0,0.5,0.6,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,1.0,0.125,0.0,0.0,0.0,0.5,1.0,0.8,0.0,1.0,0.0,0.0,1.000000,0.5,0.0,0.0,0.0,0.266667,0.0,0.0,1.0,0.666667,0.454545,1.0,0.5,0.250,0.0,0.4,0.0,0.6,0.333333,0.181818,0.666667,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.25,1.000000,0.0,0.5,0.0,0.714286,0.0,1.0,0.0,0.0,0.333333,0.0,1.0,0.5,0.0,0.833333,0.000000,0.333333,1.0,0.0,0.2,0.0,1.0,0.0,0.0
1,0.0,0.5,2.00,0.0,0.0,2.4,0.0,0.0,1.0,3.0,0.0,1.0,0.0,0.166667,0.0,4.125,1.5,2.0,0.0,1.5,0.0,2.0,0.0,0.2,5.6,0.0,0.818182,1.5,1.0,0.0,0.0,5.666667,0.0,0.0,0.0,2.333333,0.909091,0.0,2.0,0.125,0.5,1.0,3.0,2.0,1.333333,0.818182,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.6,0.000000,1.0,0.0,0.25,1.411765,0.0,1.0,0.0,1.000000,0.0,0.0,1.0,1.0,1.000000,1.0,2.0,0.0,0.0,0.166667,2.777778,2.666667,0.0,0.0,0.2,0.0,0.6,0.0,0.0
2,0.0,1.5,0.25,0.0,2.0,1.4,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.500000,1.0,2.500,1.0,0.0,2.0,0.5,0.0,2.8,1.0,0.6,1.0,0.0,1.727273,0.5,1.6,1.0,1.0,2.400000,1.0,0.0,0.0,2.285714,2.090909,0.0,1.5,0.625,1.5,0.6,0.0,3.2,2.333333,1.727273,3.000000,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.2,1.000000,2.0,1.0,1.75,2.647059,0.0,0.0,0.0,0.857143,1.0,0.5,0.0,0.0,2.333333,0.0,1.2,1.5,1.0,2.666667,2.888889,4.166667,0.0,1.0,3.0,1.0,1.2,1.0,0.0
3,0.0,0.0,0.50,0.0,1.0,1.2,1.0,1.0,1.0,0.5,0.0,0.0,0.0,0.000000,0.0,1.375,0.5,0.0,0.0,0.0,0.0,1.4,1.0,0.2,0.4,1.0,1.181818,0.0,2.0,0.0,0.0,1.066667,0.0,1.0,0.0,1.714286,1.545455,0.0,2.0,1.000,0.0,1.2,2.0,1.4,0.666667,1.454545,1.666667,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.2,0.666667,0.0,0.5,1.25,1.058824,0.0,1.5,0.0,0.571429,0.0,0.5,0.0,0.0,0.666667,0.0,1.0,0.0,0.0,1.333333,1.000000,1.333333,0.0,0.0,1.0,0.0,0.3,0.0,1.0
4,0.0,0.5,0.75,0.0,0.5,0.4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.375,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,1.000000,0.0,0.2,0.0,0.0,0.600000,0.0,0.0,1.0,0.523810,0.454545,0.0,0.0,0.375,1.0,0.0,0.0,0.6,0.000000,0.090909,0.666667,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.333333,0.0,0.0,0.00,0.764706,1.0,0.5,1.0,1.000000,0.0,0.0,0.0,0.0,0.333333,0.0,0.2,0.0,0.0,0.333333,0.111111,0.000000,0.0,0.0,0.2,0.0,0.0,0.0,0.0


In [32]:
df_rest[df_rest.index==0].sum()[df_rest[df_rest.index==0].sum()>0].sort_values(ascending=False)

Vietnamese Restaurant            1.000000
Portuguese Restaurant            1.000000
American Restaurant              1.000000
Arepa Restaurant                 1.000000
Austrian Restaurant              1.000000
Cantonese Restaurant             1.000000
Caucasian Restaurant             1.000000
Empanada Restaurant              1.000000
Falafel Restaurant               1.000000
French Restaurant                1.000000
Israeli Restaurant               1.000000
Jewish Restaurant                1.000000
North Indian Restaurant          1.000000
Afghan Restaurant                1.000000
Udon Restaurant                  1.000000
Restaurant                       1.000000
Sushi Restaurant                 1.000000
South American Restaurant        1.000000
Tapas Restaurant                 0.833333
English Restaurant               0.800000
Seafood Restaurant               0.714286
Modern European Restaurant       0.666667
Italian Restaurant               0.666667
Mediterranean Restaurant         0

In [33]:
df_rest[df_rest.index==1].sum()[df_rest[df_rest.index==1].sum()>0].sort_values(ascending=False)

Indian Restaurant                  5.666667
Fast Food Restaurant               5.600000
Chinese Restaurant                 4.125000
Brazilian Restaurant               3.000000
Malay Restaurant                   3.000000
Thai Restaurant                    2.777778
Turkish Restaurant                 2.666667
Asian Restaurant                   2.400000
Italian Restaurant                 2.333333
American Restaurant                2.000000
Sushi Restaurant                   2.000000
Doner Restaurant                   2.000000
Mediterranean Restaurant           2.000000
Kebab Restaurant                   2.000000
English Restaurant                 2.000000
Eastern European Restaurant        1.500000
German Restaurant                  1.500000
Dim Sum Restaurant                 1.500000
Restaurant                         1.411765
Mexican Restaurant                 1.333333
Greek Restaurant                   1.000000
Southern / Soul Food Restaurant    1.000000
Belgian Restaurant              

In [34]:
df_rest[df_rest.index==2].sum()[df_rest[df_rest.index==2].sum()>0].sort_values(ascending=False)

Turkish Restaurant               4.166667
Mediterranean Restaurant         3.200000
Modern European Restaurant       3.000000
Vegetarian / Vegan Restaurant    3.000000
Thai Restaurant                  2.888889
English Restaurant               2.800000
Tapas Restaurant                 2.666667
Restaurant                       2.647059
Chinese Restaurant               2.500000
Indian Restaurant                2.400000
Spanish Restaurant               2.333333
Mexican Restaurant               2.333333
Italian Restaurant               2.285714
Japanese Restaurant              2.090909
Cantonese Restaurant             2.000000
Dumpling Restaurant              2.000000
Brazilian Restaurant             2.000000
Moroccan Restaurant              2.000000
Polish Restaurant                2.000000
Argentinian Restaurant           2.000000
Ramen Restaurant                 1.750000
French Restaurant                1.727273
Middle Eastern Restaurant        1.727273
Greek Restaurant                 1

In [35]:
df_rest[df_rest.index==3].sum()[df_rest[df_rest.index==3].sum()>0].sort_values(ascending=False)

Malay Restaurant                 2.000000
Greek Restaurant                 2.000000
Kebab Restaurant                 2.000000
Italian Restaurant               1.714286
Modern European Restaurant       1.666667
Japanese Restaurant              1.545455
Scandinavian Restaurant          1.500000
Middle Eastern Restaurant        1.454545
Mediterranean Restaurant         1.400000
English Restaurant               1.400000
Chinese Restaurant               1.375000
Turkish Restaurant               1.333333
Tapas Restaurant                 1.333333
Ramen Restaurant                 1.250000
Asian Restaurant                 1.200000
Lebanese Restaurant              1.200000
Persian Restaurant               1.200000
French Restaurant                1.181818
Indian Restaurant                1.066667
Restaurant                       1.058824
Ethiopian Restaurant             1.000000
Filipino Restaurant              1.000000
Argentinian Restaurant           1.000000
Belgian Restaurant               1

In [36]:
df_rest[df_rest.index==4].sum()[df_rest[df_rest.index==4].sum()>0].sort_values(ascending=False)

Russian Restaurant                 1.000000
Molecular Gastronomy Restaurant    1.000000
Israeli Restaurant                 1.000000
Cantonese Restaurant               1.000000
Seafood Restaurant                 1.000000
Scottish Restaurant                1.000000
French Restaurant                  1.000000
Latin American Restaurant          1.000000
Pakistani Restaurant               1.000000
North Indian Restaurant            1.000000
Restaurant                         0.764706
American Restaurant                0.750000
Modern European Restaurant         0.666667
Mediterranean Restaurant           0.600000
Indian Restaurant                  0.600000
Italian Restaurant                 0.523810
Argentinian Restaurant             0.500000
African Restaurant                 0.500000
Scandinavian Restaurant            0.500000
Japanese Restaurant                0.454545
English Restaurant                 0.400000
Asian Restaurant                   0.400000
Korean Restaurant               